collect | sequence | between
recursive dates generation

In [2]:
import findspark
findspark.init()

In [13]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime


In [5]:
sc = SparkSession.builder.appName(name="master").config("master", "local").getOrCreate()

25/03/25 16:52:41 WARN Utils: Your hostname, shasankperiwal resolves to a loopback address: 127.0.1.1; using 192.168.20.45 instead (on interface wlp0s20f3)
25/03/25 16:52:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/25 16:52:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
schema = StructType([
    StructField("product_id", IntegerType()),
    StructField("period_start", DateType()),
    StructField("period_end", DateType()),
    StructField("average_daily_sales", IntegerType()),
])

data = [
    (1,datetime.strptime('2019-01-25','%Y-%m-%d').date(),datetime.strptime('2019-02-28','%Y-%m-%d').date(),100),
    (2,datetime.strptime('2018-12-01','%Y-%m-%d').date(),datetime.strptime('2020-01-01','%Y-%m-%d').date(),10),
    (3,datetime.strptime('2019-12-01','%Y-%m-%d').date(),datetime.strptime('2020-01-31','%Y-%m-%d').date(),1)
]

df = sc.createDataFrame(schema=schema, data=data)
df.show()

+----------+------------+----------+-------------------+
|product_id|period_start|period_end|average_daily_sales|
+----------+------------+----------+-------------------+
|         1|  2019-01-25|2019-02-28|                100|
|         2|  2018-12-01|2020-01-01|                 10|
|         3|  2019-12-01|2020-01-31|                  1|
+----------+------------+----------+-------------------+



Find yearly sales of each product

In [27]:
start_date = df.select("period_start").agg(min(col("period_start"))).collect()[0][0]
end_date = df.select("period_end").agg(max(col("period_end"))).collect()[0][0]
print(start_date)
print(end_date)

2018-12-01
2020-01-31


In [32]:
dates_df = sc.createDataFrame([(start_date, end_date)], ["start_date", "end_date"])
dates_df.show()
all_dates_df = dates_df.select(explode(sequence(col("start_date"), col("end_date"))).alias("start_date"))
# all_dates_df.show()

+----------+----------+
|start_date|  end_date|
+----------+----------+
|2018-12-01|2020-01-31|
+----------+----------+



In [43]:
result_df = all_dates_df.alias("a").join(df.alias("b"), how="left", on=(col("a.start_date").between(col("b.period_start"), col("b.period_end"))))\
                .select(col("product_id"), year(col("start_date")).alias("year"), col("average_daily_sales"))\
                .groupBy("product_id", "year").agg(sum("average_daily_sales").alias("avg_sales_per_product_per_year"))\
                .orderBy("product_id", "year")\
                .show()

+----------+----+------------------------------+
|product_id|year|avg_sales_per_product_per_year|
+----------+----+------------------------------+
|         1|2019|                          3500|
|         2|2018|                           310|
|         2|2019|                          3650|
|         2|2020|                            10|
|         3|2019|                            31|
|         3|2020|                            31|
+----------+----+------------------------------+

